## 文本向量化三种方法：
## 将文本分割为单词，并将每个单词转换为一个向量。
## 将文本分割为字符，并将每个字符转换为一个向量。
## 提取单词或字符的 n-gram，并将每个 n-gram 转换为一个向量。
### n-gram 是多个连续单词或字符的集合（n-gram 之间可重叠）。如：{"The", "The cat", "cat", "cat sat", "The cat sat"},为3-gram的集合（无序），称为三元语法袋，这种分词法叫词袋法
## 对文本进行分割得到的单词/字符/n-gram称为标记，对句子分解的过程叫分词
## 文本向量化即先分词，再将每个标记转换为向量

## 将向量与标记关联的方法有：one-hot、标记嵌入（通常用于词，叫词嵌入）等

## 单词级别的one-hot编码

In [2]:
import numpy as np

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
token_index = {}
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1
max_length = 10
results = np.zeros(shape=(len(samples),
                          max_length,
                          max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

In [4]:
token_index

{'The': 1,
 'cat': 2,
 'sat': 3,
 'on': 4,
 'the': 5,
 'mat.': 6,
 'dog': 7,
 'ate': 8,
 'my': 9,
 'homework.': 10}

## 字符级别的one-hot编码

In [28]:
import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
characters = string.printable
# 不确定是否正确
token_index = dict(zip(characters, range(1, len(characters) + 1)))
max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample):
        index = token_index.get(character)
        results[i, j, index] = 1.

In [30]:
results[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [32]:
## keras自带的单词级别one-hot编码
from tensorflow.keras.preprocessing.text import Tokenizer
samples = ['The cat sat on the mat.', 'The dog ate my homework.']
# num_words=1000，只保留前1000个最常见的单词
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

2022-04-24 23:20:40.985408: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


Found 9 unique tokens.


In [34]:
sequences

[[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]

## 当数据量过大时，可以采用one-hot的变体：one-hot散列，即哈希映射

## one-hot得到的很多是稀疏的，纬度高
## 而词嵌入是低维的浮点数向量（即密集向量，与稀疏向量相对），与 one-hot 编码得到的词向量不同，词嵌入是从数据中学习得到的。常见的词向量维度是 256、512 或 1024（处理非常大的词表时）。与此相对，onehot 编码的词向量维度通常为 20 000 或更高（对应包含 20 000 个标记的词表）。因此，词向量可以将更多的信息塞入更低的维度中。

## 获取词嵌入的两种方法：
- 在完成主任务（比如文档分类或情感预测）的同时学习词嵌入。在这种情况下，一开始
是随机的词向量，然后对这些词向量进行学习，其学习方式与学习神经网络的权重相同。
- 在不同于待解决问题的机器学习任务上预计算好词嵌入，然后将其加载到模型中。这些
词嵌入叫作预训练词嵌入（pretrained word embedding）。

In [35]:
# embedding层
from tensorflow.keras.layers import Embedding
# 1000：词向量最大索引+1
# 64：嵌入纬度
embedding_layer = Embedding(1000, 64)
# embedding可以看作高级的词典
# embedding输入的数据每个长度要一致，因此不够补0，太长截断

## 加载IMDB数据

In [2]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras import preprocessing
max_features = 10000
maxlen = 20
# maxlen个单词后截断
(x_train, y_train), (x_test, y_test) = imdb.load_data(
 num_words=max_features)
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/home/zy/.local/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/home/zy/.local/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you m

## 使用embedding进行情感分类

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding
model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen))
# Flatten会降为一纬
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()
history = model.fit(x_train, y_train,
 epochs=10,
 batch_size=32,
 validation_split=0.2)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 8)             80000     
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


2022-04-25 15:41:52.055284: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


625/625 [==============================] - 2s 4ms/step - loss: 0.6655 - acc: 0.6328 - val_loss: 0.6101 - val_acc: 0.7034
Epoch 2/10
625/625 [==============================] - 2s 3ms/step - loss: 0.5365 - acc: 0.7531 - val_loss: 0.5227 - val_acc: 0.7298
Epoch 3/10
625/625 [==============================] - 2s 3ms/step - loss: 0.4588 - acc: 0.7888 - val_loss: 0.5008 - val_acc: 0.7470
Epoch 4/10
625/625 [==============================] - 2s 3ms/step - loss: 0.4184 - acc: 0.8105 - val_loss: 0.4926 - val_acc: 0.7522
Epoch 5/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3900 - acc: 0.8250 - val_loss: 0.4939 - val_acc: 0.7502
Epoch 6/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3664 - acc: 0.8405 - val_loss: 0.4987 - val_acc: 0.7508
Epoch 7/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3456 - acc: 0.8523 - val_loss: 0.5072 - val_acc: 0.7530
Epoch 8/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3270 - acc:

## 使用预训练的词嵌入
## Embedding层使用预训练的词嵌入数据库，比如word2vec、GloVe（词表示全局向量）